#Getting Dataset and Keras

In [1]:
!git clone https://github.com/anubhavmaity/Sports-Type-Classifier

Cloning into 'Sports-Type-Classifier'...
remote: Enumerating objects: 14521, done.
remote: Counting objects: 100% (14521/14521), done.
remote: Compressing objects: 100% (14512/14512), done.
remote: Total 14521 (delta 7), reused 14519 (delta 5), pack-reused 0
Receiving objects: 100% (14521/14521), 592.88 MiB | 39.78 MiB/s, done.
Resolving deltas: 100% (7/7), done.
Checking out files: 100% (14619/14619), done.


In [2]:
!pip install -U git+https://github.com/keras-team/keras git+https://github.com/keras-team/keras-applications

  Cloning https://github.com/keras-team/keras to /tmp/pip-req-build-7vnspwwp
  Running command git clone -q https://github.com/keras-team/keras /tmp/pip-req-build-7vnspwwp
  Cloning https://github.com/keras-team/keras-applications to /tmp/pip-req-build-koxa5nsa
  Running command git clone -q https://github.com/keras-team/keras-applications /tmp/pip-req-build-koxa5nsa
  Stored in directory: /tmp/pip-ephem-wheel-cache-drm21hzs/wheels/18/59/26/2a3c8d65212670e9526dcb6966eba15ee401e814aa74ca121d
Successfully built Keras
  Found existing installation: Keras 2.2.4
    Uninstalling Keras-2.2.4:
      Successfully uninstalled Keras-2.2.4


#Keras Modelling

Importing keras modules

In [3]:
import keras
from keras.models import Model
from keras.layers import Input, Conv2D, MaxPooling2D, GlobalAveragePooling2D, Dense
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint

Using TensorFlow backend.


Image Data Generator

In [4]:
Generator = keras.preprocessing.image.ImageDataGenerator(#rotation_range=25,
                                                        #width_shift_range=0.2,
                                                        #height_shift_range=0.2,
                                                        #shear_range=0.2,
                                                        #zoom_range=0.2,
                                                        #fill_mode='nearest',
                                                        horizontal_flip=True,
                                                        #vertical_flip=False,
                                                        validation_split = 0.1)
shape = 224
bs = 32
classes = ['football','cricket','swimming']
train_flow = Generator.flow_from_directory('Sports-Type-Classifier/data/', target_size=(shape, shape),
                                           color_mode='rgb',
                                           class_mode='categorical', batch_size=bs,
                                           shuffle=True,
                                           classes = classes,
                                           subset = 'training')
val_flow = Generator.flow_from_directory('Sports-Type-Classifier/data/', target_size=(shape, shape),
                                           color_mode='rgb',
                                           class_mode='categorical', batch_size=bs,
                                           shuffle=True,
                                           classes = classes,
                                           subset = 'validation')

Found 1920 images belonging to 3 classes.
Found 212 images belonging to 3 classes.


Model using transfer learning on Resnet101

In [5]:
base_model = keras.applications.resnet_v2.ResNet101V2(include_top=True, weights='imagenet', input_shape=(224,224,3), classes=1000)

"""
for i,layer in enumerate(base_model.layers):
    print(i,layer.name)
"""
x = base_model.output
predictions = Dense(3, activation='softmax')(x)

model = Model( inputs = base_model.input, outputs = predictions)
#model.summary()

W0717 07:08:52.067730 139866307049344 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0717 07:08:52.107432 139866307049344 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:529: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0717 07:08:52.117657 139866307049344 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4420: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0717 07:08:52.146157 139866307049344 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4255: The name tf.nn.max_pool is deprecated. Please use tf.nn.max_pool2d instead.

W0717 07:08:52.167239 139866307049344 deprecation_wrapp

179519488/179518384 [==============================] - 6s 0us/step


Model Compilation and Callbacks

In [0]:
model.compile(loss=keras.losses.categorical_crossentropy, optimizer=Adam(lr = 0.01), metrics=['acc'])

In [0]:
save = keras.callbacks.ModelCheckpoint('resnetv2_3_classes.h5', monitor='val_loss', verbose=1, save_best_only=True, save_weights_only=True, mode='auto', period=1)
lr = keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=2, verbose=1, mode='auto', min_delta=0.0001, cooldown=0, min_lr=0.00001)

In [9]:
model.fit_generator(
        train_flow,
        steps_per_epoch=1920//32,
        epochs=30,
        validation_data=val_flow,
        validation_steps=212//32,
        callbacks = [save,lr])

W0717 07:09:46.932595 139866307049344 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Epoch 1/30
60/60 [==============================] - 111s 2s/step - loss: 1.0053 - acc: 0.5339 - val_loss: 1.2750 - val_acc: 0.3177

Epoch 00001: val_loss improved from inf to 1.27502, saving model to resnetv2_3_classes.h5
Epoch 2/30
60/60 [==============================] - 85s 1s/step - loss: 0.9243 - acc: 0.5547 - val_loss: 0.9131 - val_acc: 0.5444

Epoch 00002: val_loss improved from 1.27502 to 0.91308, saving model to resnetv2_3_classes.h5
Epoch 3/30
60/60 [==============================] - 85s 1s/step - loss: 0.7581 - acc: 0.6339 - val_loss: 0.6175 - val_acc: 0.7000

Epoch 00003: val_loss improved from 0.91308 to 0.61749, saving model to resnetv2_3_classes.h5
Epoch 4/30
60/60 [==============================] - 85s 1s/step - loss: 0.6966 - acc: 0.6474 - val_loss: 0.7390 - val_acc: 0.6111

Epoch 00004: val_loss did not improve from 0.61749
Epoch 5/30
60/60 [==============================] - 85s 1s/step - loss: 0.6718 - acc: 0.6542 - val_loss: 0.5992 - val_acc: 0.6556

Epoch 00005: va

In [0]:
model.load_weights('resnetv2_3_classes.h5')

#Predicting Video Category

In [0]:
import os
import cv2
import numpy as np

In [0]:
def predict(video_path,model,classes):
    #################################
    '''
    INPUT-
    
    video_path -- path to the video to be predicted
    
    model -- the trained model to be used for prediction
    
    classes -- list of classes that model was trained on
    
    OUTPUT-
    
    All frames stored to data/ folder
    
    predicted class of every frame in a numpy array
    '''
    #################################
    cam = cv2.VideoCapture(video_path)
    os.makedirs('data')
    # frame 
    currentframe = 0
      
    while 1:
        #print(0)
        # reading from frame 
        ret,frame = cam.read()   
        if ret: 
            # if video is still left continue creating images 
            name = './data/frame' + str(currentframe) + '.jpg'
            #print ('Creating...' + name)
            # Cropping the extracted image
            frame = cv2.resize(frame,(shape,shape))
            # writing the extracted images 
            cv2.imwrite(name, frame)   
            # increasing counter so that it will 
            # show how many frames are created 
            currentframe += 1
        else: 
            break  
    # Release all space and windows once done 
    cam.release() 
    cv2.destroyAllWindows()
    
    # Store all frame paths in a list
    
    #Data_generator for frames
    pred_gen = keras.preprocessing.image.ImageDataGenerator()
    pred_flow = pred_gen.flow_from_directory('', target_size=(shape, shape),color_mode='rgb',class_mode='categorical', batch_size=bs,shuffle=False, classes = ['data'])
    
    pred = model.predict_generator(pred_flow)
    #the pred is probability of classes getting the max using argmax
    pred = np.argmax(pred,axis = 1)
    #getting the maximum occuring class across all frames
    counts = np.bincount(pred)
    print(classes[np.argmax(counts)])
    return pred

In [0]:
!rm -r data

In [49]:
pred_frame_wise = predict('30 Seconds of Premier League skills - HD.mp4',model = model,classes = classes)

Found 912 images belonging to 1 classes.
football


In [0]:
def save_vid_with_class(pred_frame_wise,classes):
    #################################
    '''
    INPUT-
    
    pred_frame_wise -- numpy array of predicted class number of every frame
        
    classes -- list of classes that model was trained on
    
    OUTPUT-
    
    Video with class of every frame stored to disk as Output_video.avi
    
    '''
    #################################    
    output = cv2.imread('data/frame0.jpg')
    height, width, layers = output.shape
    size = (width,height)
    fourcc = cv2.VideoWriter_fourcc(*"DIVX")
    writer = cv2.VideoWriter("Output_video.avi", fourcc, 30, size)
    
    for i in range(pred_frame_wise.shape[0]):
        output = cv2.imread('data/frame' + str(i) + '.jpg')
        # draw the activity on the output frame
        text = "activity: {}".format(classes[pred_frame_wise[i]])
        cv2.putText(output, text, (10, 20), cv2.FONT_HERSHEY_SIMPLEX,
        	0.5, (25, 0, 190), 2)
        
        # write the output frame to disk
        writer.write(output)
    writer.release

In [0]:
save_vid_with_class(pred_frame_wise,classes)